In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Import API key
from config import api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

Generate Cities List

In [4]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

616

In [6]:
#perform weather check on each city using a series of successive API calls
#include a print log of each city as it's being processed with city number and city name

count = 1
err = 0
sets = 0
index = 0
t0 = time.time()

print(f"Beginning Data Retrieval \n ...............")
for city in cities:
    try:
        city1 = city.replace(" ", "%20")
        print(f"Processing Record {count} of Set {sets} | {city_name}" )
        print(f"{api.openweathermap.org/data/2.5/weather?id={city1}&appid={api_key}}")
        

SyntaxError: unexpected EOF while parsing (<ipython-input-6-a6899cf12c84>, line 16)